# Lab 1

In [1]:
# All code below is referenced from Lecture_1_2.ipynb provided by Gittu George for DSCI 525

In [2]:
# import packages

import re
import os
import glob
import zipfile
import requests
from urllib.request import urlretrieve
import json
import pandas as pd
from memory_profiler import memory_usage
from os import listdir
from functools import reduce

In [3]:
%load_ext rpy2.ipython

In [4]:
%load_ext memory_profiler

In [5]:
# Necessary metadata
article_id = 14096681  # this is the unique identifier of the article on figshare
url = f"https://api.figshare.com/v2/articles/{article_id}"
headers = {"Content-Type": "application/json"}
output_directory = "figshareairline/"

In [6]:
response = requests.request("GET", url, headers=headers)
data = json.loads(response.text)  # this contains all the articles data, feel free to check it out
files = data["files"]             # this is just the data about the files, which is what we want
files

[{'is_link_only': False,
  'name': 'daily_rainfall_2014.png',
  'supplied_md5': 'fd32a2ffde300a31f8d63b1825d47e5e',
  'computed_md5': 'fd32a2ffde300a31f8d63b1825d47e5e',
  'id': 26579150,
  'download_url': 'https://ndownloader.figshare.com/files/26579150',
  'size': 58863},
 {'is_link_only': False,
  'name': 'environment.yml',
  'supplied_md5': '060b2020017eed93a1ee7dd8c65b2f34',
  'computed_md5': '060b2020017eed93a1ee7dd8c65b2f34',
  'id': 26579171,
  'download_url': 'https://ndownloader.figshare.com/files/26579171',
  'size': 192},
 {'is_link_only': False,
  'name': 'README.md',
  'supplied_md5': '61858c6cc0e6a6d6663a7e4c75bbd88c',
  'computed_md5': '61858c6cc0e6a6d6663a7e4c75bbd88c',
  'id': 26586554,
  'download_url': 'https://ndownloader.figshare.com/files/26586554',
  'size': 5422},
 {'is_link_only': False,
  'name': 'data.zip',
  'supplied_md5': 'b517383f76e77bd03755a63a8ff83ee9',
  'computed_md5': 'b517383f76e77bd03755a63a8ff83ee9',
  'id': 26766812,
  'download_url': 'https://

Next, we download the data:

In [7]:
# make directory if missing
os.makedirs(output_directory, exist_ok=True)

# download missing files
files_to_dl = ["data.zip"]
for item in filter(lambda x: x['name'] in files_to_dl, files):
    filename = os.path.join(output_directory, item["name"])
    if not os.path.isfile(filename):
        urlretrieve(item["download_url"], filename)

The download took 55 mins on Raf's computer.

In [8]:
# open a read-only connection to zip file
zfile = zipfile.ZipFile(glob.glob(output_directory + "*.zip")[0], "r")

# list non-hidden files in zip
csvs = list(filter(lambda x: not x.startswith("__"), zfile.namelist()))

In [9]:
%%time
%%memit

# create a dictionary of dataframes
dat = {x.split('_daily')[0]: pd.read_csv(zfile.open(x)) for x in csvs}

# reshape to one big dataframe
dat = pd.concat(dat, names=["model", "row"])

peak memory: 10401.11 MiB, increment: 10247.26 MiB
CPU times: user 1min 11s, sys: 6.6 s, total: 1min 18s
Wall time: 1min 19s


Unfortunately, `pd.concat` is quite memory hungry, as it will make a copy along the way. I tried a while loop with a `dat.pop(0)` but I haven't figured out how to avoid memory leaks and/or key errors.

In [10]:
print(dat.shape)

(62513863, 6)


In [11]:
dat.head()

time    lat_min    lat_max   lon_min  \
model           row                                                        
MPI-ESM-1-2-HAM 0    1889-01-01 12:00:00 -35.439867 -33.574619  141.5625   
                1    1889-01-02 12:00:00 -35.439867 -33.574619  141.5625   
                2    1889-01-03 12:00:00 -35.439867 -33.574619  141.5625   
                3    1889-01-04 12:00:00 -35.439867 -33.574619  141.5625   
                4    1889-01-05 12:00:00 -35.439867 -33.574619  141.5625   

                      lon_max  rain (mm/day)  
model           row                           
MPI-ESM-1-2-HAM 0    143.4375   4.244226e-13  
                1    143.4375   4.217326e-13  
                2    143.4375   4.498125e-13  
                3    143.4375   4.251282e-13  
                4    143.4375   4.270161e-13